In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
sns.set(style="white",color_codes=True)

In [2]:
HouseDS=pd.read_csv("HousePS.csv")
HouseDS.head()

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000.0,6,3,2.0,10,West,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000.0,5,3,2.0,9,East,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,Beautiful To Floor House : House For Sale In D...,"Dhapasi, Dhapasi, Kathmandu",Kathmandu,70000000.0,5,3,2.0,12,East,154,1-1-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,10-1-0-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,Modern Architecture Bungalow House : House For...,"baluwatar, Baluwatar, Kathmandu",Kathmandu,130000000.0,6,4,3.0,9,West,17,0-10-1-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-7-1-0 Aana,1 day ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,Modern Bungalow House : House For Sale In Golf...,"Golfutar, Golfutar, Kathmandu",Kathmandu,100000000.0,6,3,2.0,10,East,59,0-19-2-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-2-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


In [3]:
HouseDS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       2211 non-null   object 
 1   Address     2211 non-null   object 
 2   City        2211 non-null   object 
 3   Price       2211 non-null   float64
 4   Bedroom     2211 non-null   int64  
 5   Bathroom    2211 non-null   int64  
 6   Floors      1039 non-null   float64
 7   Parking     2211 non-null   int64  
 8   Face        2211 non-null   object 
 9   Views       2211 non-null   object 
 10  Area        2211 non-null   object 
 11  Road        2211 non-null   object 
 12  Road Width  2211 non-null   object 
 13  Road Type   1426 non-null   object 
 14  Build Area  2211 non-null   object 
 15  Posted      2211 non-null   object 
 16  Amenities   2211 non-null   object 
dtypes: float64(2), int64(3), object(12)
memory usage: 293.8+ KB


In [4]:
HouseDS.describe()

,Price,Bedroom,Bathroom,Floors,Parking
count,2.211000e+03,2211.000000,2211.000000,1039.000000,2211.000000
mean,4.065559e+11,2.731796,1.871099,2.824206,1.007689
std,7.566414e+12,4.174364,2.635836,1.265411,2.094905
min,1.500000e+01,0.000000,0.000000,0.000000,0.000000
25%,1.300000e+06,0.000000,0.000000,2.000000,0.000000
50%,8.000000e+06,0.000000,0.000000,3.000000,0.000000
75%,2.650000e+07,5.000000,4.000000,3.000000,1.000000
max,2.160000e+14,95.000000,40.000000,13.000000,31.000000


In [5]:
HouseDS["Price"].value_counts()

Price
3500000.0     35
25000000.0    33
2500000.0     32
15000000.0    30
35000000.0    29
              ..
24000.0        1
35200000.0     1
96000000.0     1
12000.0        1
59000000.0     1
Name: count, Length: 407, dtype: int64

In [6]:
#splitting Data set into Training and testing sets
X=HouseDS.drop(['Price','Title','Address','City','Face'],axis=1)
y=HouseDS['Price']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [7]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1768, 12) (443, 12) (1768,) (443,)


In [8]:
#feature scaling(to normalize feature variables)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
#X_train_scale=scaler.fit_transform(X_train)
#X_test_scale=scaler.transform(X_test)
#Scaling string ko laagi navayera pending vayo

In [10]:
svm_model=SVR(kernel='linear')

In [11]:
#Training the model
svm_model.fit(X_train_scale,y_train)

NameError: name 'X_train_scale' is not defined

In [ ]:
#Making predictions on test set
pred=svm_model.predict(X_test_scale)

In [ ]:
#Evaluating the model
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test,pred)
mse